In [ ]:
import pandas as pd
import zipfile
import os
import tarfile
import math
import numpy as np
import os

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [ ]:
data_route = './data/dgt/';

# Preprocesado de los archivos directamente descargados de la DGT

Una vez descargados los archivos de transacciones de la página de microdatos de la DGT, es requerido un procesamiento previo de los datos, debido a su codificación (ISO-8859-1) y a su formato, ya que no se trata de un csv.

Los archivos de la DGT son zip's que contienen archivos de ancho fijo. La diferencia con respecto a los csv's es que no son están separados por, valga la redundancia, separadores, sino que cada campo tiene un tamaño de caracteres máximo asignado.

A continuación se irán explicando las diferentes funciones que aparecen en este notebook y su función

La función "gbk_to_utf8" se encarga de abrir el archivo de ancho fijo de la DGT y cambiar su codificación

In [ ]:
def gbk_to_utf8(input_file, output_file):
    # Load Files
    input_file_opened = open(input_file, 'r', encoding='ISO-8859-1')
    input_file_read = input_file_opened.read()
    input_file_opened.close()
    os.remove(input_file)
    
    output_file_opened = open(output_file, 'x', encoding='utf-8', newline='\n')
    # Transcode
    print('---- Transcoding…')
    output_file_opened.write(input_file_read)
    output_file_opened.close()
    
    print('---- Transcoding Done')

"extract_dataframe_dgt_tourist" sirve como una función que llama a otros procedimientos y, además, lee el archivo de ancho fijo y lo carga como Dataframe de Pandas

In [ ]:
def extract_dataframe_dgt_tourist(route_files, file_zip):
    
    widths = [8,1,8,30,22,1,21,2,1,5,6,6,6,3,2,2,2,2,24,2,2,1,8,5,8,1,1,9,3,5,30,7,3,5,1,1,1,1,1,1,11,25,25,35,70,6,6,4,4,3,8,4,4,4,6,30,50,35,25,35,4,4,4,1,25,1,4,25,8]
    columns = ['FEC_MATRICULA','COD_CLASE_MAT','FEC_TRAMITACION','MARCA_ITV','MODELO_ITV','COD_PROCEDENCIA_ITV','BASTIDOR_ITV','COD_TIPO','COD_PROPULSION_ITV','CILINDRADA_ITV','POTENCIA_ITV','TARA','PESO_MAX','NUM_PLAZAS','IND_PRECINTO','IND_EMBARGO','NUM_TRANSMISIONES','NUM_TITULARES','LOCALIDAD_VEHICULO','COD_PROVINCIA_VEH','COD_PROVINCIA_MAT','CLAVE_TRAMITE','FEC_TRAMITE','CODIGO_POSTAL','FEC_PRIM_MATRICULACION','IND_NUEVO_USADO','PERSONA_FISICA_JURIDICA','CODIGO_ITV','SERVICIO','COD_MUNICIPIO_INE_VEH','MUNICIPIO','KW_ITV','NUM_PLAZAS_MAX','CO2_ITV','RENTING','COD_TUTELA','COD_POSESION','IND_BAJA_DEF','IND_BAJA_TEMP','IND_SUSTRACCION','BAJA_TELEMATICA','TIPO_ITV','VARIANTE_ITV','VERSION_ITV','FABRICANTE_ITV','MASA_ORDEN_MARCHA_ITV','MASA_MÁXIMA_TECNICA_ADMISIBLE_ITV','CATEGORÍA_HOMOLOGACIÓN_EUROPEA_ITV','CARROCERIA','PLAZAS_PIE','NIVEL_EMISIONES_EURO_ITV','CONSUMO_WH/KM_ITV','CLASIFICACIÓN_REGLAMENTO_VEHICULOS_ITV','CATEGORÍA_VEHÍCULO_ELÉCTRICO','AUTONOMÍA_VEHÍCULO_ELÉCTRICO','MARCA_VEHÍCULO_BASE','FABRICANTE_VEHÍCULO_BASE','TIPO_VEHÍCULO_BASE','VARIANTE_VEHÍCULO_BASE','VERSIÓN_VEHÍCULO_BASE','DISTANCIA_EJES_12','VIA_ANTERIOR_ITV','VIA_POSTERIOR_ITV','TIPO_ALIMENTACION_ITV','CONTRASEÑA_HOMOLOGACION_ITV','ECO_INNOVACION_ITV','REDUCCION_ECO_ITV','CODIGO_ECO_ITV','FEC_PROCESO']
    tourist_code = '40'

    print('-- Decompress file…')

    with zipfile.ZipFile(route_files+file_zip,"r") as zip_ref:
        zip_ref.extractall(path=route_files, members=None, pwd=None)
        
    file_txt = zip_ref.namelist()[0]
    
    print('-- Change encode to UTF-8')
    gbk_to_utf8(route_files+file_txt, route_files+file_txt)
    
    print('-- Transform to dataframe…')
    dgt_df = pd.read_fwf(route_files+file_txt, widths = widths, header=None)
    dgt_df.columns = columns
    
    os.remove(route_files+file_txt)
    
    dgt_df.replace('^\s+', '', regex=True, inplace=True) #front
    dgt_df.replace('\s+$', '', regex=True, inplace=True) #end
    
    print('-- Extract tourist rows')
    dgt_df = dgt_df[dgt_df['COD_TIPO'] == tourist_code].reset_index()
    
    return dgt_df
    
    
    

La función "create_csv_tar_gz" tiene como objetivo almacenar el Dataframe como un csv y luego comprimirlo en formato "tar.gz"

In [ ]:
def create_csv_tar_gz(year):

    dgt_df = create_dataset_tourist_year(year)
    
    csv_file = "export_anual_trf_"+year+".csv"
    
    tar_file = "export_anual_trf_"+year+".csv.tar.gz"

    dgt_df.to_csv(data_route+csv_file, index=False, sep="^")
    
    cmd = 'tar -czvf '+data_route+tar_file+' '+data_route+csv_file

    os.system(cmd)
    
    os.remove(data_route+csv_file)


La siguiente función reduce el Dataframe a unos determinados campos, ya que muchos de los campos que contiene el archivo de la DGT con informativos y se ha comprobado que, o no tienen relevancia o que contienen una proporción muy elevada de Nan

In [ ]:
def reduce_fields(data_set):    
    return data_set[['FEC_MATRICULA','COD_CLASE_MAT','FEC_TRAMITACION','MARCA_ITV','MODELO_ITV','COD_PROCEDENCIA_ITV','COD_PROPULSION_ITV','CILINDRADA_ITV','POTENCIA_ITV','NUM_PLAZAS','NUM_TRANSMISIONES','NUM_TITULARES','LOCALIDAD_VEHICULO','COD_PROVINCIA_VEH','COD_PROVINCIA_MAT','CLAVE_TRAMITE','FEC_TRAMITE','CODIGO_POSTAL','FEC_PRIM_MATRICULACION','IND_NUEVO_USADO','PERSONA_FISICA_JURIDICA','COD_MUNICIPIO_INE_VEH','MUNICIPIO','KW_ITV','NUM_PLAZAS_MAX','CO2_ITV','RENTING','CATEGORÍA_HOMOLOGACIÓN_EUROPEA_ITV','NIVEL_EMISIONES_EURO_ITV','CONSUMO_WH/KM_ITV','CATEGORÍA_VEHÍCULO_ELÉCTRICO','AUTONOMÍA_VEHÍCULO_ELÉCTRICO']]

"month_to_quarter" calcula el cuatrimestre del año para meterlo como campo dentro del Dataframe

In [ ]:
def month_to_quarter(month):
    if (month >= 1 and month <=12):
        return (month-1)//3 + 1
    else:
        return np.nan



"create_new_fields" recalcula campos e inserta nuevos que serán de utilidad para el estudio del Dataframe

In [ ]:
def create_new_fields(df):
    
    print('-- START format dates and create new fields')
    df['FEC_MATRICULA'] = df['FEC_MATRICULA'].apply(lambda x: pd.to_datetime(str(x)[-4:]+'-'+str(x)[-6:-4]+'-'+str(x)[:-6]) )
    df['FEC_TRAMITACION'] = df['FEC_TRAMITACION'].apply(lambda x: (pd.to_datetime(str(int(x))[-4:]+'-'+str(int(x))[-6:-4]+'-'+str(int(x))[:-6])) if not math.isnan(x) else x )
    df['FEC_TRAMITE'] = df['FEC_TRAMITE'].apply(lambda x: pd.to_datetime(str(x)[-4:]+'-'+str(x)[-6:-4]+'-'+str(x)[:-6]) )
    # df['FEC_PROCESO'] = df['FEC_PROCESO'].apply(lambda x: pd.to_datetime(str(x)[-4:]+'-'+str(x)[-6:-4]+'-'+str(x)[:-6]) )
    df['year_Q'] = df['FEC_TRAMITE'].apply(lambda x: x.strftime("%Y") )
    df['Q_query'] = df['FEC_TRAMITE'].apply(lambda x: month_to_quarter(int(x.strftime("%m"))) )
    df['FEC_PRIM_MATRICULACION'] = df['FEC_PRIM_MATRICULACION'].apply(lambda x: (pd.to_datetime(str(int(x))[-4:]+'-'+str(int(x))[-6:-4]+'-'+str(int(x))[:-6])) if not math.isnan(x) else x )
    print('-- END format dates and create new fields')
    
    return df
    

"create_dataset_tourist_year" va uniendo los diferentes Dataframe. La DGT tiene separados los archivos por meses, pero para su almacenamiento, se realizará en años

In [ ]:
def create_dataset_tourist_year(year):

    dgt_df = False

    for subdir, dirs, files in os.walk(data_route):
        for file in sorted(files):
            if ( file.find('.zip') != -1 and file.find(year) != -1):
                print(file)

                df_aux = extract_dataframe_dgt_tourist(data_route, file)
                df_aux = reduce_fields(df_aux)
                df_aux = create_new_fields(df_aux)

                if (type(dgt_df) == pd.core.frame.DataFrame):
                    dgt_df = pd.concat([dgt_df,df_aux], keys=None)
                else:
                    dgt_df = df_aux.copy()
                    
    return dgt_df

Y, como punto final, se realiza la ejecución de todo el proceso, limitado a entre los años 2015 y 2021

In [ ]:
for year in ['2015', '2016', '2017','2018','2019','2020','2021']:
    print(year)
    create_csv_tar_gz(year)